# Clean and merge data

**This section cleans and merges activity data from BigQuery and investment portfolio data from MySQL into one data frame.**

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from targetandmarket.config import data_folder

In [22]:
# Import app activity data
# Note that session numbers and engagement times have been scaled by a hidden factor to obfuscate true values
user_analytics_raw = pd.read_csv(data_folder/'user_analytics_raw.csv', index_col=0)
user_error = pd.read_csv(data_folder/'user_error_status.csv', index_col=0)
timeseries = pd.read_csv(data_folder/'timeseries.csv', index_col=0)

**Let's look at the activity data which I queried from BigQuery**

In [3]:
print(user_analytics_raw.shape)

(173344, 6)


In [4]:
user_analytics_raw.head()

,user_id,sessions,last_session,total_time,avg_session,first_open
0,129682562,65,1.590957e+09,5236.317,80.558723,1.524686e+09
1,201385452,41,1.590787e+09,2354.798,57.434098,1.588205e+09
2,1558824692,79,1.590962e+09,9315.622,117.919266,1.580936e+09
3,118978210,75,1.590777e+09,5122.445,68.299267,1.531505e+09
4,701136272,30,1.590686e+09,8831.959,294.398633,1.579424e+09


**I have queried:**
* user_id: obfuscated user ID
* sessions: total number of sessions
* last_session: unix timestamp of last session (seconds)
* total_time: total engagement time
* avg_session: average session time
* first_open: unix timestamp of when app was first opened

**Session numbers were absent for some users so any user which such a session is removed**

In [5]:
user_analytics = user_analytics_raw.loc[~user_analytics_raw['user_id'].
                                        isin(user_error['user_id'])].copy()
print(user_analytics.shape)

(172493, 6)


**Since I am interested in predicting which users are paying users based on activity from May 2020, I need to exclude those users that started using the app in the middle of May 2020**

In [6]:
# Users who started using the app since app launch and 01 May 2020 are selected
PERIOD_START = pd.Timestamp('2017-11-01').timestamp()
PERIOD_END = pd.Timestamp('2020-05-01').timestamp()
user_analytics = user_analytics.loc[
    (user_analytics['first_open'] >= PERIOD_START)
    & (user_analytics['first_open'] < PERIOD_END)].copy()
print('Number of users between analysis duration:', len(user_analytics))

Number of users between analysis duration: 135674


**Converting unix timestamps to "delta" between June 01, 2020 (in terms of number of days ago)**

In [7]:
LAST_MODIFIED = 1590969600
user_analytics.loc[:, 'last_session'] = (
    (LAST_MODIFIED - user_analytics['last_session']) / (60 * 60 * 24))
user_analytics.loc[:, 'first_open'] = (
    (LAST_MODIFIED - user_analytics['first_open']) / (60 * 60 * 24))
user_analytics = user_analytics.astype('float64')

**Calculating the number of days a user was active in May 2020 from timeseries data**

In [8]:
timeseries = timeseries.pivot(index='user_id',
                              columns='date',
                              values='day_eng_time')
active_days = pd.notna(timeseries).sum(axis=1)
active_days.name = 'active_days'

user_analytics = pd.merge(user_analytics,
                          active_days,
                          how='inner',
                          on='user_id')

In [9]:
user_analytics.head()

,user_id,sessions,last_session,total_time,avg_session,first_open,active_days
0,1.296826e+08,65.0,0.149734,5236.317,80.558723,767.166667,29
1,2.013855e+08,41.0,2.115601,2354.798,57.434098,32.000000,21
2,1.558825e+09,79.0,0.087216,9315.622,117.919266,116.125000,28
3,1.189782e+08,75.0,2.224022,5122.445,68.299267,688.250000,24
4,7.011363e+08,30.0,3.279432,8831.959,294.398633,133.625000,24


**Now let's look at the investment data and our label (free vs paying user)**

In [10]:
user_holdings = pd.read_csv(data_folder/'user_holdings.csv')
user_purchases = pd.read_csv(data_folder/'user_purchases_may.csv', index_col=0)
user_purchases_june = pd.read_csv(data_folder/'user_purchases_early_june.csv', index_col=0)
user_transactions = pd.read_csv(data_folder/'user_transactions.tsv', sep='\t')

In [11]:
user_transactions.head()

,obfuscatedId,isPro,status,numberOfTransactions,numberOfCoins,numberOfCoinIds
0,671119916,1,ACTIVE,138,25,3
1,936520911,0,ACTIVE,4,2,2
2,44359534,1,ACTIVE,3,2,0
3,399029160,1,ACTIVE,236,20,9
4,2000673413,0,ACTIVE,3,3,0


In [12]:
#%% Merge number of transactions and number of coins with user analytics
user_analytics = pd.merge(user_analytics,
                          user_transactions[['obfuscatedId', 'numberOfTransactions', 'numberOfCoins']],
                          how='inner',
                          left_on='user_id',
                          right_on='obfuscatedId')
user_analytics = user_analytics.drop(['obfuscatedId'], axis=1)

In [13]:
user_holdings.head()

,obfuscatedId,isPro,status,holdings
0,671119916,1,ACTIVE,5.691166e+04
1,44359534,1,ACTIVE,2.117894e+05
2,399029160,1,ACTIVE,6.593942e+06
3,1123126564,0,ACTIVE,2.902064e+03
4,585372673,1,ACTIVE,4.058743e+04


In [14]:
user_holdings.describe()

,obfuscatedId,isPro,holdings
count,6.649160e+05,664916.000000,6.649160e+05
mean,1.073004e+09,0.010317,1.223419e+272
std,6.199896e+08,0.101048,inf
min,1.468000e+03,0.000000,-4.156190e+259
25%,5.362605e+08,0.000000,7.963817e+01
50%,1.072384e+09,0.000000,4.739244e+02
75%,1.610624e+09,0.000000,2.810146e+03
max,2.147482e+09,1.000000,8.134710e+277


**There are some outliers when it comes to holdings (value of investment portfolio). There are some users with negative holdings and some with holdings greater than the market capitalization of all digital assets. So I will only keep users with more than 10 USD value and less than 10 million USD**

In [15]:
user_holdings = user_holdings.loc[(user_holdings.holdings < 10000000)
                                  & (user_holdings.holdings > 10)]

In [16]:
# Merge with activity data
user_analytics = pd.merge(user_analytics,
                          user_holdings[['obfuscatedId', 'holdings', 'isPro']],
                          how='inner',
                          left_on='user_id',
                          right_on='obfuscatedId')
user_analytics = user_analytics.drop(['obfuscatedId'], axis=1)

**The isPro column indicates which user is currently a "PRO" or premium user. It also includes users who are on free trial (7 days) so these are actually free users and I need to change the label to 0**

In [17]:
# Purchase history from May 2020
user_purchases = user_purchases.dropna()
user_purchases = user_purchases.drop(['event_name'], axis=1)

mapping = {
    'io.getdelta.android.delta_pro_yearly_trial': 0,
    'io.getdelta.ios.DELTA_PRO_EARLY_BACKER_MONTHLY': 1,
    'io.getdelta.ios.DELTA_PRO_YEARLY_TRIAL': 0,
    'io.getdelta.ios.DELTA_PRO_EARLY_BACKER_MONTHLY_EQUALIZED': 1,
    'io.getdelta.ios.DELTA_PRO_EARLY_BACKER_YEARLY': 1,
    'io.getdelta.android.delta_pro_early_backer_yearly_equalized': 1
}

user_purchases.loc[:, 'product_id'] = user_purchases['product_id'].map(mapping)
user_purchases = user_purchases.sort_values(by=['user_id', 'product_id'])
# Keep last product "purchased" because those that actually purchased premium after trial should be left as is
user_purchases = user_purchases.drop_duplicates(subset=['user_id'],
                                                keep='last')
user_analytics = pd.merge(user_analytics,
                          user_purchases,
                          how='left',
                          on='user_id')
# Users that have pro status = 1 but are on trial (product_id ==0), set pro status as 0
user_analytics.loc[((user_analytics.isPro == 1) &
                    (user_analytics.product_id == 0)), 'isPro'] = 0
user_analytics = user_analytics.drop(['product_id'], axis=1)

**The isPro status is as of June 03, 2020. Due to the way the company stores data, historical status of isPro is not available. Therefore, I have used purchase history on June 01, 02 and 03 to set users that purchased premium during these days as non-premium for May 2020**

In [18]:
user_purchases_june = user_purchases_june.drop(['event_name'], axis=1)
user_purchases_june = user_purchases_june.dropna()
user_purchases_june.loc[:, 'product_id'] = user_purchases_june['product_id'].map(mapping)
user_purchases_june = user_purchases_june.sort_values(by=['user_id', 'product_id'])
user_purchases_june = user_purchases_june.drop_duplicates(subset=['user_id'], keep='last')
user_analytics = pd.merge(user_analytics, user_purchases_june, how='left', on='user_id')
# Set pro status for those who purchased or tried after June 01 as 0
user_analytics.loc[((user_analytics.isPro==1) & (~pd.isna(user_analytics.product_id))), 'isPro'] = 0
user_analytics = user_analytics.drop(['product_id'], axis=1)

In [19]:
user_analytics.head()

,user_id,sessions,last_session,total_time,avg_session,first_open,active_days,numberOfTransactions,numberOfCoins,holdings,isPro
0,1.296826e+08,65.0,0.149734,5236.317,80.558723,767.166667,29,34,24,8778.301834,0
1,1.558825e+09,79.0,0.087216,9315.622,117.919266,116.125000,28,118,20,2739.913695,0
2,1.189782e+08,75.0,2.224022,5122.445,68.299267,688.250000,24,11,10,354.697827,0
3,1.691281e+09,26.0,0.184756,1791.426,68.901000,99.041667,19,119,18,4000.690597,0
4,4.059957e+08,195.0,0.095354,13298.463,68.197246,376.416667,31,1038,14,178797.427788,1


In [20]:
#%% Save processed data
user_analytics.to_csv(data_folder/'user_analytics.csv')